In [575]:
from __future__ import division
from ps4_utils import load_data,load_experiment
from ps4_utils import AbstractGenerativeModel
from ps4_utils import save_submission
from scipy.misc import logsumexp
#from sklearn.metrics import confusion_matrix
import numpy as np
#data_fn = "datasets-hw4.h5"
data_fn = "datasets-hw4-1.h5"

MAX_OUTER_ITER = 15

In [576]:
class MixtureModel(AbstractGenerativeModel):
    def __init__(self, CLASSES, NUM_FEATURES, NUM_MIXTURE_COMPONENTS, MAX_ITER=50, EPS=10**(-7)):
        AbstractGenerativeModel.__init__(self, CLASSES, NUM_FEATURES)
        self.num_mixture_components = NUM_MIXTURE_COMPONENTS # list of num_mixture_components (length num_classes)
        self.max_iter = MAX_ITER # max iterations of EM
        self.epsilon = EPS # help with stability, to be used according to hint given at end of pset4.pdf
        self.params = { # lists of length CLASSES
            'pi': [np.repeat(1/k,k) for k in self.num_mixture_components], # with pi_c for each class
            'theta': [np.zeros((self.num_features,k)) for k in self.num_mixture_components], # with theta_c for each class
        }
        
    def pack_params(self, X, class_idx):
        pi,theta = self.fit(X[class_idx],class_idx) # fit parameters
        self.params['pi'][class_idx] = pi # update member variable pi
        self.params['theta'][class_idx] = theta #update member variable theta
        
    #make classification based on which mixture model gives higher probability to generating point xi
    def classify(self, X):
        P = list()
        pi = self.params['pi']
        theta = self.params['theta']
        for c in range(self.num_classes):
            _,Pc = self.findP(X, pi[c], theta[c])
            P.append(Pc)
        return np.vstack(P).T.argmax(-1) # np.array of class predictions for each data point in X

    # --- E-step
    def updateLatentPosterior(self, X, pi, theta, num_mixture_components): # update the latent posterior
        # YOUR CODE HERE
        # --- gamma: responsibilities (probabilities), np.array (matrix)
        # ---        shape: number of data points in X (where X consists of datapoints from class c) by NUM_MIXTURE_COMPONENTS[c]
        # note: can use output of findP here (with care taken to return gamma containing proper probabilities)
        #N = X.shape[0]
        #gamma = np.ones(N,num_mixture_components) #theta.shape[1]
        t,logsumt = self.findP(X,pi,theta)

        T = np.repeat(logsumt, num_mixture_components).reshape((-1, num_mixture_components))
        """
        T = np.ones((t.shape[0],t.shape[1]))
        for i in range(t.shape[0]):
            for j in range(t.shape[1]):
                T[i][j] = logsumt[i]
        """
        gamma = np.exp(t-T)
           
        return gamma
    # --- M-step (1)
    @staticmethod
    def updatePi(gamma): #update the pi component using the posteriors (gammas)
        # YOUR CODE HERE
        # --- pi_c: class specific pi, np.array (vector)
        # ---        shape: NUM_MIXTURE_COMPONENTS[c]

        pi_c = np.sum(gamma, axis=0)/gamma.shape[0]
        return pi_c
    # -- M-step (2)
    @staticmethod
    def updateTheta(X, gamma): #update theta component using posteriors (gammas)
        # YOUR CODE HERE
        # --- theta_c: class specific theta, np.array matrix
        # ---        shape: NUM_FEATURES by NUM_MIXTURE_COMPONENTS[c]
        Nk = np.sum(gamma, axis=0)
        matrix = np.dot(np.transpose(X), gamma)
        theta_c = matrix/Nk
        return theta_c 
    
    @staticmethod
    def findP(X, pi, theta):
        # YOUR CODE HERE
        # NOTE: you can also use t as a probability, just change "logsumexp(t,axis=1)" to "logsumexp(np.log(t),axis=1)"
        # --- t: logprobabilities of x given each component of mixture
        # ---        shape: number of data points in X (where X consists of datapoints from class c) by NUM_MIXTURE_COMPONENTS[c] 
        # --- logsumexp(t,axis=1): (for convenience) once exponentiated, gives normalization factor over all mixture components
        # ---        shape: number of data points in X (where X consists of datapoints from class c)
        
        N = X.shape[0]
        d = X.shape[1]
        
        t = np.ones([N,theta.shape[1]]) 
                    
        
        for c in range(pi.shape[0]):
            pi_c = pi[c]
            for i in range(0, N):
                theta_cj = theta[:,c]
                prob = np.log(pi_c)+ np.sum((np.log(theta_cj)*X[i,:]) + (np.log(1-theta_cj)*(1-X[i,:])))
                t[i][c] = prob #np.log(pi_c)+
        """       
        for c in range(pi.shape[0]):
            pi_c = pi[c]
            for i in range(0, N):
                prob = 0
                for j in range (0, d):
                    theta_cj = theta[j][c]
                    prob = prob + (np.log(theta_cj)*X[i][j]) + (np.log(1-theta_cj)*(1-X[i][j]))
                t[i][c] = np.log(pi_c)+prob
                
        
        for i in range(0, N):
            for c in range(theta.shape[1]):
                prob = 0
                pi_c = pi[c]
                for j in range (0, d):
                    theta_cj = theta[j][c]
                    prob = prob + (np.log(theta_cj)*X[i][j]) + (np.log(1-theta_cj)*(1-X[i][j]))
                t[i][c] = np.log(pi_c)+prob 
        """
        return t,logsumexp(t,axis=1)
        
    # --- execute EM procedure
    def fit(self, X, class_idx):
        max_iter = self.max_iter
        eps = self.epsilon
        N = X.shape[0]
        pi = self.params['pi'][class_idx]
        theta = self.params['theta'][class_idx]
        num_mixture_components = self.num_mixture_components[class_idx]
        # INITIALIZE theta, note theta is currently set to zeros but needs to be officially initialized here
        for i in range(num_mixture_components):
            theta[:,i] = (np.sum(X[range(i,N,num_mixture_components),:], axis=0) + self.epsilon) / np.size(X[range(i,N,num_mixture_components),:],0)
        
        for i in range(theta.shape[0]):
            for j in range(theta.shape[1]):   
                if theta[i][j] < eps:
                    theta[i][j] = eps
                elif theta[i][j] > 1-eps:
                    theta[i][j] = 1-eps
                    
        for i in range(max_iter):
            # YOUR CODE HERE, E-step: 
            gamma = self.updateLatentPosterior(X, pi, theta, num_mixture_components)
            # YOUR CODE HERE, M-step(1): 
            pi = self.updatePi(gamma)
            # YOUR CODE HERE, M-step(2): 
            theta = self.updateTheta(X,gamma)
            
            for i in range(theta.shape[0]):
                for j in range(theta.shape[1]):   
                    if theta[i][j] < eps:
                        theta[i][j] = eps
                    elif theta[i][j] > 1-eps:
                        theta[i][j] = 1-eps
        
        return pi,theta #pi and theta, given class_idx

In [577]:
class NaiveBayesModel(AbstractGenerativeModel):
    def __init__(self, CLASSES, NUM_FEATURES, EPS=10**(-12)):
        AbstractGenerativeModel.__init__(self, CLASSES, NUM_FEATURES)
        self.epsilon = EPS # help with stability
        self.params = {
            'p': [np.zeros((NUM_FEATURES))] * self.num_classes # estimated log-probabilities of features for each class
        }
    def pack_params(self, X, class_idx):
        p = self.fit(X[class_idx])
        self.params['p'][class_idx] = p
    def classify(self, X): # naive bayes classifier
        # YOUR CODE HERE
        # --- predictions: predictions for data points in X (where X consists of datapoints from class c), np.array (vector)
        # ---        shape: number of data points
        N = X.shape[0]
        d = X.shape[1]
        predictions = np.zeros(N)
        prob = np.ones((self.num_classes,N))

        for c in range(self.num_classes):
            for i in range(0, N):
                row = X[i]
                for j in range (0, d):
                    theta = self.params['p'][c][j]
                    prob[c][i] = prob[c][i] * (theta**row[j])*((1-theta)**(1-row[j]))

        predictions = prob.argmax(axis=0)
        return predictions
    def fit(self, X):
        # YOUR CODE HERE
        # --- estimated_p: estimated p's of features for input X (where X consists of datapoints from class c), np.array (vector)
        # ---        shape: NUM_FEATURES
        estimated_p = np.zeros(X.shape[1])
        sum_arr = np.sum(X, axis=0)
        for i in range(0, X.shape[1]):
            estimated_p[i] = (sum_arr[i]+self.epsilon)/X.shape[0]
        return estimated_p

In [578]:
experiment_name = "sentiment_analysis"
# --- SENTIMENT ANALYSIS setup
Xtrain,Xval,num_classes,num_features = load_experiment(data_fn, experiment_name)

# -- build naive bayes model for sentiment analysis
print("SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:")
nbm = NaiveBayesModel(num_classes, num_features)
nbm.train(Xtrain)
print("ACCURACY ON VALIDATION: " + str(nbm.val(Xval)))

print("CONFUSION MATRIX: ")
cmatrix = np.zeros((num_classes, num_classes))
for actual_c in range(num_classes):
    classes = nbm.classify(Xval[actual_c]) 
    for predict_c in classes:
        cmatrix[predict_c][actual_c] += 1
print(cmatrix)

# -- build mixture model for sentiment analysis
print("SENTIMENT ANALYSIS -- MIXTURE MODEL:")
for i in range(MAX_OUTER_ITER):
    num_mixture_components =  np.random.randint(2,15,num_classes)
    print("COMPONENTS: " + " ".join(str(i) for i in num_mixture_components))
    mm = MixtureModel(num_classes, num_features, num_mixture_components)
    mm.train(Xtrain)
    print("ACCURACY ON VALIDATION: " + str(mm.val(Xval)))

# submit to kaggle
Xkaggle = load_data(data_fn, experiment_name, "kaggle")
save_submission("mm-{}-submission.csv".format(experiment_name), mm.classify(Xkaggle))

SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.738
CONFUSION MATRIX: 
[[  94.   60.]
 [  71.  275.]]
SENTIMENT ANALYSIS -- MIXTURE MODEL:
COMPONENTS: 2 4
ACCURACY ON VALIDATION: 0.712
COMPONENTS: 4 12
ACCURACY ON VALIDATION: 0.704
COMPONENTS: 13 13
ACCURACY ON VALIDATION: 0.738
COMPONENTS: 5 9
ACCURACY ON VALIDATION: 0.714
COMPONENTS: 6 12
ACCURACY ON VALIDATION: 0.694
COMPONENTS: 12 3
ACCURACY ON VALIDATION: 0.696
COMPONENTS: 14 9
ACCURACY ON VALIDATION: 0.72
COMPONENTS: 13 10
ACCURACY ON VALIDATION: 0.716
COMPONENTS: 7 13
ACCURACY ON VALIDATION: 0.71
COMPONENTS: 2 7
ACCURACY ON VALIDATION: 0.696
COMPONENTS: 10 3
ACCURACY ON VALIDATION: 0.71
COMPONENTS: 14 10
ACCURACY ON VALIDATION: 0.72
COMPONENTS: 11 7
ACCURACY ON VALIDATION: 0.726
COMPONENTS: 13 8
ACCURACY ON VALIDATION: 0.716
COMPONENTS: 7 2
ACCURACY ON VALIDATION: 0.71
('Saved:', 'mm-sentiment_analysis-submission.csv')


In [579]:
experiment_name = "mnist"
# --- MNIST DIGIT CLASSIFICATION setup
Xtrain,Xval,num_classes,num_features = load_experiment(data_fn, experiment_name)

# -- build naive bayes model for mnist digit classification
print("MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:")
nbm = NaiveBayesModel(num_classes, num_features)
nbm.train(Xtrain)
print("ACCURACY ON VALIDATION: " + str(nbm.val(Xval)))

print("CONFUSION MATRIX: ")
cmatrix = np.zeros((num_classes, num_classes))
for actual_c in range(num_classes):
    classes = nbm.classify(Xval[actual_c]) 
    for predict_c in classes:
        cmatrix[predict_c][actual_c] += 1
print(cmatrix)

# -- build mixture model for mnist digit classification
print("MNIST DIGIT CLASSIFICATION -- MIXTURE MODEL:")
for i in range(MAX_OUTER_ITER):
    num_mixture_components =  np.random.randint(2,15,num_classes)
    print("COMPONENTS: " + " ".join(str(i) for i in num_mixture_components))
    mm = MixtureModel(num_classes, num_features, num_mixture_components)
    mm.train(Xtrain)
    print("ACCURACY ON VALIDATION: " + str(mm.val(Xval)))
    
# submit to kaggle
Xkaggle = load_data(data_fn, experiment_name, "kaggle")
save_submission("mm-{}-submission.csv".format(experiment_name), mm.classify(Xkaggle))

MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.7355
CONFUSION MATRIX: 
[[ 151.    0.    5.    3.    2.    7.    2.    1.    2.    1.]
 [   0.  206.   14.    7.    3.   12.    8.   13.   18.    5.]
 [   5.    5.  147.   10.    5.    3.    6.    2.   13.    3.]
 [   5.    1.   11.  137.    0.   31.    2.    3.   20.   10.]
 [   0.    2.    6.    0.  147.    9.    9.   12.    5.   22.]
 [  10.    3.    2.    8.    4.   91.    9.    1.    8.    5.]
 [  11.    1.    9.    2.    2.    5.  177.    0.    3.    1.]
 [   1.    2.    3.    4.    6.    6.    1.  179.    0.   21.]
 [   2.    2.    9.    8.    3.    6.    2.    2.  122.    4.]
 [   2.    1.    4.    4.   21.    2.    1.   20.    5.  114.]]
MNIST DIGIT CLASSIFICATION -- MIXTURE MODEL:
COMPONENTS: 2 3 7 13 13 8 14 6 5 4
ACCURACY ON VALIDATION: 0.766
COMPONENTS: 5 2 5 3 3 7 12 10 10 14
ACCURACY ON VALIDATION: 0.7775
COMPONENTS: 14 4 8 6 7 9 9 9 7 7
ACCURACY ON VALIDATION: 0.7815
COMPONENTS: 7 4 6 9 13 7 11 5

PROBLEM 1:
x-axis = predicted
y-axis = actual

SENTIMENT ANALYSIS -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.738
CONFUSION MATRIX: 
[[  94.   60.]
 [  71.  275.]]

MNIST DIGIT CLASSIFICATION -- NAIVE BAYES MODEL:
ACCURACY ON VALIDATION: 0.7355
CONFUSION MATRIX: 
[[ 151.    0.    5.    3.    2.    7.    2.    1.    2.    1.]
 [   0.  206.   14.    7.    3.   12.    8.   13.   18.    5.]
 [   5.    5.  147.   10.    5.    3.    6.    2.   13.    3.]
 [   5.    1.   11.  137.    0.   31.    2.    3.   20.   10.]
 [   0.    2.    6.    0.  147.    9.    9.   12.    5.   22.]
 [  10.    3.    2.    8.    4.   91.    9.    1.    8.    5.]
 [  11.    1.    9.    2.    2.    5.  177.    0.    3.    1.]
 [   1.    2.    3.    4.    6.    6.    1.  179.    0.   21.]
 [   2.    2.    9.    8.    3.    6.    2.    2.  122.    4.]
 [   2.    1.    4.    4.   21.    2.    1.   20.    5.  114.]]